In [4]:
# # Required Packages for the Project RUN ONLY ONCE
# %pip install matplotlib
# %pip install nilearn
# %pip install openpyxl
# %pip install Path
# %pip install seaborn
# %pip install nltools
# %pip install scipy
# %pip install scikit-image
# %pip install nibabel
# %pip install bctpy
# %pip install statsmodels

In [55]:
# import copy
# from skimage import filters
# import json
# import nibabel as nib
# from nilearn.plotting import plot_img

import pandas as pd
from nilearn import image
from nilearn.image import load_img, index_img
import numpy as np
import scipy.io
from scipy import stats
import networkx as nx
import matplotlib as mt
import scipy as sc
import seaborn as sns
import bct
from copy import deepcopy
from statsmodels.stats import multitest

In [43]:
class SubjectData:
    
    def __init__(self, datasetPath, componentsPath):
        self.componentFileName = "adni_aa__sub01_component_ica_s1_.nii"
        self.timeCourseFileName = "adni_aa__sub01_timecourses_ica_s1_.nii"
        self.FNCmatFile = "adni_aa__postprocess_results.mat"
        self.component_key = "fnc_corrs_all"
        self.graphMetricGlobalMeasues = [
                # Global Measures
                "Global efficiency", 
                "Characteristic path length", 
                "Clustering coefficient",
            ]
        self.graphMetricComponentMeasures = [
                # Component Measures
                "Degree",
                "Closeness centrality",
                "Participation coefficient"
            ]

        self.dictionary = {
            "AD": "Alzhiemeris",
            "CN": "Controls",
            "MCI": "Mild Cognitive Impairment"
        }
        
        self.subjectsData = self.readFileofCSV(datasetPath)
        self.componentData = self.readFileofCSV(componentsPath)

        self.modifySubjectsPath()

        self.domainList = self.componentData["icn_domain"]
        self.indexList = self.componentData["icn_index"]

    def modifySubjectsPath(self):
        for i in range(2404):
            self.subjectsData.at[i,"fc_dir"] = self.subjectsData.iloc[i]["fc_dir"].replace("FC","GIGICA")

    def readFileofCSV(self,path):
        fileData = pd.read_csv(path)
        return fileData
    
    def getDatasetPaths(self,subjectName):
        ans = list()
        for i in range(2404):
            if self.subjectsData.at[i,"ResearchGroup"] == subjectName:
                ans.append(self.subjectsData.at[i,"fc_dir"])
        return ans

    def calculatePearsonCrossCorrelations(self, listA, listB):
        xmean = np.mean(listA)
        xval = listA - xmean
        xsqu = np.sqrt(np.sum(np.square(xval)))

        ymean = np.mean(listB)
        yval = listB - ymean
        ysqu = np.sqrt(np.sum(np.square(yval)))

        num = 0
        for k in range(len(yval)):
            num += (xval[k]*yval[k])

        den = (xsqu * ysqu)
        ans = num/den
        return ans

In [7]:
class VoxelCounts(SubjectData):
    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "
    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.voxelCountMap = dict()
        self.prepareVoxelCountMap()

    def prepareVoxelCountMap(self):
        " \
            iterate over the list of Subjects like AD, CN, ....     \
        "
        for subjectName in self.subjectsToCalculate:
            self.voxelCountMap[subjectName]=dict()
            self.voxelCountMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            self.voxelCountMap[subjectName]["indexes"] = dict()
            for i in self.indexList:
                self.voxelCountMap[subjectName]["indexes"][i]=list()

    def calculateVoxelCount(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.voxelCountMap[subjectName]["paths"]:
                spacialMapName = path + self.componentFileName
                spacialMap = load_img(spacialMapName)
                for index in self.voxelCountMap[subjectName]["indexes"]:
                    actualIndex=index-1
                    componentImg = index_img(spacialMap, actualIndex)
                    componentImgData = componentImg.get_fdata()
                    component_threshold = 3*np.std(componentImgData)
                    component_voxelCount = np.count_nonzero(componentImgData > component_threshold)
                    self.voxelCountMap[subjectName]["indexes"][index].append(component_voxelCount)

In [8]:

AD_Threshold = VoxelCounts('ADNI_demos.txt', 'NM_icns_info.csv', [ "CN", "MCI",  "AD" ] )
AD_Threshold.calculateVoxelCount()

In [9]:
class GraphMetrics(SubjectData):

    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "

    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.graphMetricMap = dict()
        self.prepareGraphMetricMap()

    def prepareGraphMetricMap(self):
        for subjectName in self.subjectsToCalculate:
            self.graphMetricMap[subjectName]=dict()
            self.graphMetricMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            for graphMetric in self.graphMetricGlobalMeasues:
                self.graphMetricMap[subjectName][graphMetric]=list()

            for graphMetric in self.graphMetricComponentMeasures:
                self.graphMetricMap[subjectName][graphMetric] = dict()
                for ind in self.indexList:
                    self.graphMetricMap[subjectName][graphMetric][ind]=list()

    def loadMatFile(self,filePath):
        componentDict = scipy.io.loadmat(filePath+self.FNCmatFile)
        return componentDict[self.component_key]

    def prepareFNCMatrix(self,componentData):
        selected_component = np.zeros((53,53), dtype=np.float64).reshape(53,53)
        for i in range(53):
            for j in range(i+1, 53):
                # if componentData[self.indexList[i]-1][self.indexList[j]-1] >=0 : 
                selected_component[i][j]=componentData[self.indexList[i]-1][self.indexList[j]-1]
        selected_component += selected_component.T

        # Finding correlation matrix
        corrs = pd.DataFrame(selected_component)
        correlation_matrix = corrs.corr()
        correlation_numpy_array = correlation_matrix.to_numpy(dtype=np.float64)

        # Finding Thresholded correlation matrix
        row,column = correlation_numpy_array.shape
        for i in range(row):
            for j in range(column):
                if i!=j and correlation_numpy_array[i][j]<0:
                    correlation_numpy_array[i][j]=0

        input_matrix = nx.from_numpy_array(correlation_numpy_array)
        return input_matrix

    def calculateGraphMetrics(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.graphMetricMap[subjectName]["paths"]:
                fncMatrix = self.prepareFNCMatrix(self.loadMatFile(path))

                # for global measures:
                for graphMetric in self.graphMetricGlobalMeasues:
                    if graphMetric == "Global efficiency":
                        globalEfficieny = nx.global_efficiency(fncMatrix)
                        self.graphMetricMap[subjectName][graphMetric].append(globalEfficieny)

                    elif graphMetric == "Characteristic path length":
                        characteristicPathLength = nx.average_shortest_path_length(fncMatrix, weight='weight')
                        self.graphMetricMap[subjectName][graphMetric].append(characteristicPathLength)

                    elif graphMetric == "Clustering coefficient":
                        clusterCofficient = nx.clustering(fncMatrix, weight='weight')
                        clusterCofficient_network = 0
                        for i in clusterCofficient:
                            clusterCofficient_network += clusterCofficient[i]
                        clusterCofficient_network /= 53
                        self.graphMetricMap[subjectName][graphMetric].append(clusterCofficient_network)

                    else:
                        print("Not Found: ", graphMetric)
                        assert(False)

                # for component measures
                for graphMetric in self.graphMetricComponentMeasures:
                    if graphMetric == "Degree":
                        all_nodes_degree = nx.degree(fncMatrix, weight= 'weight')
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_degree[ind])

                    elif graphMetric == "Closeness centrality":
                        all_nodes_cc = nx.closeness_centrality(fncMatrix, distance='weight', wf_improved=False)
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_cc[ind])

                    elif graphMetric == "Participation coefficient":
                        fncMatrix_numpy = nx.to_numpy_array(fncMatrix)
                        modularity = nx.algorithms.community.greedy_modularity_communities(fncMatrix)
                        sam=[]
                        for i in range(len(modularity)):
                            for j in list(modularity[i]):
                                sam.append(j)
                        sam = np.array(sam)
                        all_nodes_pc = bct.centrality.participation_coef(fncMatrix_numpy, ci=sam, degree='undirected')
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_pc[ind])

                    else:
                        print("Not Found: ", graphMetric)
                        assert(False)

In [10]:

AD_GraphMetric = GraphMetrics('ADNI_demos.txt', 'NM_icns_info.csv', [ "CN", "MCI", "AD" ])
AD_GraphMetric.calculateGraphMetrics()

In [73]:
class CorrsCorrelations:
    def __init__(self,sub1, sub2):
        self.AD_GraphMetric = sub2
        self.AD_Threshold = sub1

        self.CrossListTables = dict()
        self.PListTables = dict()
        self.FDRTables = dict()

    def calculateStatsCrossCorrelation(self):
        GlobalGraphMetrics_Names = self.AD_GraphMetric.graphMetricGlobalMeasues
        ComponentGraphMetric_Names = self.AD_GraphMetric.graphMetricComponentMeasures

        for subjectName in self.AD_GraphMetric.subjectsToCalculate:

            graphMetricsTable = pd.DataFrame()
            pvaluesTables = pd.DataFrame()
            fdrTables = pd.DataFrame()
            for i in range(len(self.AD_Threshold.indexList)):
                graphMetricsTable[self.AD_Threshold.domainList[i]+'('+str(self.AD_Threshold.indexList[i])+')'] = list()
                pvaluesTables[self.AD_Threshold.domainList[i]+'('+str(self.AD_Threshold.indexList[i])+')'] = list()
                fdrTables[self.AD_Threshold.domainList[i]+'('+str(self.AD_Threshold.indexList[i])+')'] = list()

            # For Global Measure
            for graphMetricName in GlobalGraphMetrics_Names:
                indexList = self.AD_Threshold.indexList
                corr_list_global = list()
                p_list_global = list()

                xlist_global = deepcopy(self.AD_GraphMetric.graphMetricMap[subjectName][graphMetricName])
                for j in range(len(indexList)):

                    ylist_global = deepcopy(self.AD_Threshold.voxelCountMap[subjectName]["indexes"][self.AD_Threshold.indexList[j]])
                    # corr_list_global.append(self.AD_Threshold.calculatePearsonCrossCorrelations(xlist_global, ylist_global) )
                    prs, pval = stats.pearsonr(xlist_global, ylist_global)
                    corr_list_global.append(prs)
                    p_list_global.append(round(pval, 5))

                graphMetricsTable.loc[len(graphMetricsTable.index)] = corr_list_global
                pvaluesTables.loc[len(pvaluesTables.index)] = p_list_global
                fdrTables.loc[len(fdrTables.index)] = multitest.fdrcorrection(p_list_global, 0.05)[0]


            # For Component Measure
            for i in ComponentGraphMetric_Names:
                indexList = self.AD_Threshold.indexList
                corr_list_component = list()
                p_list_component = list()

                for j in range(len(indexList)):
        
                    xlist_component = deepcopy(self.AD_GraphMetric.graphMetricMap[subjectName][i][indexList[j]])
                    ylist_component = deepcopy(self.AD_Threshold.voxelCountMap[subjectName]["indexes"][AD_Threshold.indexList[j]])
                    # corr_list_component.append(self.AD_Threshold.calculatePearsonCrossCorrelations(xlist_component, ylist_component))
                    prs, pval = stats.pearsonr(xlist_component, ylist_component)
                    corr_list_component.append(prs)
                    p_list_component.append(round(pval, 5))

                graphMetricsTable.loc[len(graphMetricsTable.index)] = corr_list_component
                pvaluesTables.loc[len(pvaluesTables.index)] = p_list_component
                fdrTables.loc[len(fdrTables.index)] = multitest.fdrcorrection(p_list_component, 0.05)[0]

            graphMetricsTable.index = self.AD_GraphMetric.graphMetricGlobalMeasues + self.AD_GraphMetric.graphMetricComponentMeasures
            pvaluesTables.index = self.AD_GraphMetric.graphMetricGlobalMeasues + self.AD_GraphMetric.graphMetricComponentMeasures
            fdrTables.index = self.AD_GraphMetric.graphMetricGlobalMeasues + self.AD_GraphMetric.graphMetricComponentMeasures
            self.CrossListTables[subjectName] = graphMetricsTable
            self.PListTables[subjectName] = pvaluesTables
            self.FDRTables[subjectName] = fdrTables

    def savePValues(self):
        for key in self.PListTables:
            table = self.PListTables[key]
            table.to_csv(key+'.csv')

    def saveFDRValues(self):
        for key in self.FDRTables:
            table = self.FDRTables[key]
            table.to_csv(key+'.csv')

In [69]:
crossCorr = CorrsCorrelations(AD_Threshold, AD_GraphMetric)
crossCorr.calculateStatsCrossCorrelation()
# crossCorr.savePValues()
crossCorr.saveFDRValues()

In [72]:
crossCorr.FDRTables['MCI']

,SC(69),SC(53),SC(98),SC(99),SC(45),AU(21),AU(56),SM(3),SM(9),SM(2),...,DM(40),DM(23),DM(71),DM(17),DM(51),DM(94),CB(13),CB(18),CB(4),CB(7)
Global efficiency,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Characteristic path length,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Clustering coefficient,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Degree,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Closeness centrality,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Participation coefficient,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [40]:
crossCorr.CrossListTables['MCI']

,SC(69),SC(53),SC(98),SC(99),SC(45),AU(21),AU(56),SM(3),SM(9),SM(2),...,DM(40),DM(23),DM(71),DM(17),DM(51),DM(94),CB(13),CB(18),CB(4),CB(7)
Global efficiency,-0.331404,-0.246248,-0.273650,-0.246876,-0.280152,-0.227475,-0.294594,-0.243687,-0.212399,-0.283213,...,-0.276220,-0.207214,-0.302819,-0.238383,-0.217005,-0.266212,-0.211652,-0.170418,-0.227672,-0.195692
Characteristic path length,0.085102,0.008566,0.035837,0.095824,-0.044255,-0.051898,0.103369,-0.031039,0.095583,0.115515,...,0.048173,0.140200,0.026671,0.106100,0.009950,-0.041960,0.023990,-0.007318,0.038639,-0.012436
Clustering coefficient,0.047891,-0.029817,-0.002082,0.059027,-0.079396,-0.042123,0.045446,-0.038848,0.039188,0.089561,...,0.022610,0.139157,-0.017623,0.066691,-0.034240,-0.031050,-0.007067,-0.013021,0.024236,-0.003141
Degree,0.238353,-0.015690,0.017157,0.026343,-0.015723,-0.150753,-0.056741,-0.062328,-0.003519,0.041774,...,0.004442,0.097530,-0.048635,0.031732,-0.096536,-0.089062,-0.108699,0.018118,-0.151998,-0.130613
Closeness centrality,0.022407,-0.005693,0.004969,0.005131,0.153762,0.077387,0.000417,0.079017,-0.007694,-0.086704,...,-0.030673,-0.074231,0.032735,0.042351,0.038280,0.024859,-0.014366,-0.045283,0.016506,0.074830
Participation coefficient,0.315826,0.078878,0.110889,0.137958,0.102398,-0.192224,-0.062080,-0.100439,-0.020188,0.012971,...,-0.061605,0.119774,-0.066954,0.098795,-0.130455,-0.121044,-0.120838,-0.053435,-0.167284,-0.133950


In [21]:
crossCorr.componentPValues[1]

,SC(69),SC(53),SC(98),SC(99),SC(45),AU(21),AU(56),SM(3),SM(9),SM(2),...,DM(40),DM(23),DM(71),DM(17),DM(51),DM(94),CB(13),CB(18),CB(4),CB(7)
Degree,"(0.23835279067662393, 3.7704356235930235e-06)","(-0.01569002091750766, 0.7641914350075896)","(0.01715663856342103, 0.7428921196590043)","(0.02634335003687878, 0.6144572652304954)","(-0.015723227389389867, 0.7637071457103087)","(-0.15075343344796488, 0.00374716404843777)","(-0.056741464103489694, 0.27762866133888303)","(-0.06232819007141077, 0.2329656272818284)","(-0.0035189424549700588, 0.9463623229807104)","(0.041774390242279844, 0.4242937329711309)",...,"(0.004442300440848923, 0.9323181545325943)","(0.09753040470442116, 0.061615783072196194)","(-0.04863490946662006, 0.35218682533152224)","(0.03173173611919416, 0.5439822110175309)","(-0.09653618511071972, 0.0643268710123497)","(-0.08906150756842984, 0.08799406183746993)","(-0.10869944085015118, 0.03713267379138513)","(0.0181184682360665, 0.7290282530049951)","(-0.15199843821793832, 0.0034676867092944346)","(-0.13061272299052895, 0.012148056453998099)"
Closeness centrality,"(0.022407179063054276, 0.6683339641754862)","(-0.005693410210875431, 0.9133231635128598)","(0.004969471221534981, 0.9243090668906219)","(0.005130996446671213, 0.9218565716717366)","(0.15376166549159734, 0.003104159439438485)","(0.0773873573680981, 0.13841502155540727)","(0.0004170759136586308, 0.9936379880274048)","(0.07901679097182479, 0.13027596409964184)","(-0.007694226503467214, 0.8830523901381419)","(-0.08670353860039565, 0.0967671913866812)",...,"(-0.030672683931212297, 0.5575124633213278)","(-0.07423112236224057, 0.15528287230456228)","(0.03273485068081665, 0.5313183943949895)","(0.042350989172461, 0.41792156479567394)","(0.03827968959453355, 0.46410637674311933)","(0.02485907203447326, 0.6345512258726154)","(-0.014366366650877992, 0.783568795894594)","(-0.0452827697106438, 0.38640064606953245)","(0.01650553217061211, 0.7523248822306132)","(0.07482974677534898, 0.1519695955970079)"
Participation coefficient,"(0.31582558728241217, 5.743939953682346e-10)","(0.07887840175364376, 0.13095251478138922)","(0.11088863380212144, 0.03345847695056708)","(0.1379575415872563, 0.00804526399217624)","(0.10239763650006939, 0.04966864685679469)","(-0.19222407291189525, 0.00020758349284321416)","(-0.062080413029090495, 0.23483205150058908)","(-0.10043898427123013, 0.054220683394703324)","(-0.02018824088567845, 0.6994971061000006)","(0.012971160924547705, 0.8041391195189825)",...,"(-0.06160461925401231, 0.23844556286166285)","(0.11977398481721442, 0.021554654573638008)","(-0.06695357490769699, 0.2000317842562898)","(0.09879503085899584, 0.05830415751058519)","(-0.13045469781542762, 0.012253667996047645)","(-0.121044362445451, 0.02019633905546801)","(-0.12083820701770676, 0.02041157571324132)","(-0.053435101914510896, 0.30663600774454264)","(-0.16728357573490027, 0.001278052860919172)","(-0.13395032824256733, 0.01009746159097397)"


In [ ]:
# temp_global = deepcopy(ans_global)
# temp_component = deepcopy(ans_component)

In [ ]:
symvalue = 0.5
fileNames = ['Controls', 'Mild_Cognetive_Impairment', 'Alzheimers']

# for i in range(len(fileNames)):
#         # temp_global[i].to_csv(fileNames[i]+'_global.csv')

#         s1 = temp_global[i].to_numpy()
#         fig, ax = mt.pyplot.subplots(figsize=(54,15))
#         columnsName = temp_global[i].columns.values
#         rowsName = temp_global[i].index.values
#         sd = sns.heatmap(
#                         s1, cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0, vmin= -1*symvalue, vmax=symvalue,
#                         xticklabels=columnsName, yticklabels=rowsName,  annot=True
#                 )
#         ax.set_title(fileNames[i]+str(symvalue)+'_global.png')
#         sd.get_figure().savefig(fileNames[i]+str(symvalue)+'_global.png')

# for i in range(len(fileNames)):
#         # temp_component[i].to_csv(fileNames[i]+'_component.csv')

#         s1 = temp_component[i].to_numpy()
#         fig, ax = mt.pyplot.subplots(figsize=(54,15))
#         columnsName = temp_component[i].columns.values
#         rowsName = temp_component[i].index.values
#         sd = sns.heatmap(
#                         s1, cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0, vmin= -1*symvalue, vmax=symvalue,
#                         xticklabels=columnsName, yticklabels=rowsName,  annot=True
#                 )
#         ax.set_title(fileNames[i]+str(symvalue)+'_component.png')
#         sd.get_figure().savefig(fileNames[i]+str(symvalue)+'_component.png')

In [ ]:
def findDifferenceCrossCorrelation(name1, name2, name3, tab1, tab2):
    diff = pd.DataFrame().reindex_like(tab1).dropna()
    columns = tab1.columns.values

    rows = tab1.index.values

    for i in rows:
        for j in columns:
            diff.at[i,j] = tab1.at[i,j] - tab2.at[i,j]

    st1 = diff.to_numpy()

    fig, ax = mt.pyplot.subplots(figsize=(54,15))

    sd = sns.heatmap (
                    st1, 
                    vmin= -1*symvalue, 
                    vmax= symvalue,
                    cmap='coolwarm', 
                    square=True,
                    ax=ax,
                    linewidth='0.5',
                    center=0, 
                    cbar=True,
                    xticklabels=columns, 
                    yticklabels=rows,  
                    annot=True
                )
    ax.set_title(name1+'-'+name2+'-'+name3+str(symvalue)+'.png')
    sd.get_figure().savefig(name1+'-'+name2+'-'+name3+str(symvalue)+'.png')


In [ ]:
sizeN = len(fileNames)

# for i in range(sizeN):
#     for j in range(i+1,sizeN):
#         findDifferenceCrossCorrelation(fileNames[i], fileNames[j], 'global', temp_global[i], temp_global[j])
#         findDifferenceCrossCorrelation(fileNames[i], fileNames[j], 'component', temp_component[i], temp_component[j])

In [ ]:
import os
import random
from PIL import Image, ImageOps

def concat_images(image_paths, size, shape=None):
    # Open images and resize them
    width, height = size
    images = map(Image.open, image_paths)
    images = [ImageOps.fit(image, size, Image.ANTIALIAS) 
              for image in images]
    
    # Create canvas for the final image with total size
    shape = shape if shape else (1, len(images))
    image_size = (width * shape[1], height * shape[0])
    image = Image.new('RGB', image_size)
    
    # Paste images into final image
    for row in range(shape[0]):
        for col in range(shape[1]):
            offset = width * col, height * row
            idx = row * shape[1] + col
            image.paste(images[idx], offset)
    
    return image

def createImage(keyword):
    # Get list of image paths
    folder = '.'
    for key in keyword:
        image_array = [ i+str(symvalue)+'_'+key+'.png' for i in fileNames ]
        for i in range(sizeN):
            for j in range(i+1,sizeN):
                image_array.append(fileNames[i]+'-'+fileNames[j]+'-'+key+str(symvalue)+'.png')
        image = concat_images(image_array, (3888,1080), (3,2))
        image.save('image_'+key+'.png', 'PNG')

# createImage(['global', 'component'])

In [ ]:
"""
1 4d subject
    - 53 selected Components
        - each component voxel Count - index i

AD- 213
69  - [ 1_Sub_69Ind_voxelCount, 2_Sub_69Ind_VoxelCount,  ]
53  - [ ]

1 4D subject:
    - preprocess.mat file 
        - FNC matrix
            - Graph metric 
                - Glo
                - Charac
                - Coefficint

AD- 213
Global Efficieny - [ 1_Sub_mat_FNC_Global_Value, 2_Sub_mat_FNC_Global, ..... ]
Characterstic Path lenght - [ 1_sub, ........ ]


Correlation:
Global Efficiency Vs 69 [] - Correlation 0.123
Global Efficiency Vs 53 [] - Correlation -0.324
                                            0.372


"""